## 概要

> Stable Baselines, a Fork of OpenAI Baselines - Training, Saving and Loading

学習したモデルを保存して，それを読み込み再度学習を始めれる環境を作っていきます．

Github Repo: [https://github.com/hill-a/stable-baselines](https://github.com/hill-a/stable-baselines)

Medium article: [https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82](https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82)

>[RL Baselines Zoo](https://github.com/araffin/rl-baselines-zoo) is a collection of pre-trained Reinforcement Learning agents using Stable-Baselines.

RL Baselines Zooは、Stable-Baselinesを用いた強化学習エージェントをあらかじめ学習させたコレクションです。

>It also provides basic scripts for training, evaluating agents, tuning hyperparameters and recording videos.

また、トレーニング、エージェントの評価、ハイパーパラメータの調整、ビデオの録画のための基本的なスクリプトが用意されています。


Documentation is available online: [https://stable-baselines.readthedocs.io/](https://stable-baselines.readthedocs.io/)



## パッケージのインストール

公式のドキュメントは古く，そのままでは動かないため，下記のように少し修正します，

### OpenGym関係

ほんとに毎回Verに悩まされます

In [ ]:
!pip install "gym==0.19.0"
#!pip install stable-baselines[mpi]
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines[mpi]==2.10.2 box2d box2d-kengz

### tensorflow

In [ ]:
!pip uninstall -y tensorflow-gpu
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0

### tensorboard

In [ ]:
!pip uninstall tensorboard-plugin-wit --yes

## Import policy, RL agent, ...

In [ ]:
import gym
import numpy as np

from stable_baselines import DQN

## Create the Gym env and instantiate the agent

>For this example, we will use Lunar Lander environment.

この例では、Lunar Lander環境を使用します。

>"Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine. "

着陸パッドの外に着陸することも可能です。燃料は無限であるため、エージェントは飛行を学習し、最初の試みで着陸することができます。何もしない」「左旋回エンジン起動」「メインエンジン起動」「右旋回エンジン起動」の4つの個別アクションが可能です。

>Lunar Lander environment: [https://gym.openai.com/envs/LunarLander-v2/](https://gym.openai.com/envs/LunarLander-v2/)

![Lunar Lander](https://cdn-images-1.medium.com/max/960/1*f4VZPKOI0PYNWiwt0la0Rg.gif)

>Note: vectorized environments allow to easily multiprocess training. In this example, we are using only one process, hence the DummyVecEnv.

注意：ベクトル化された環境では、簡単にマルチプロセスで学習することができます。この例では、1つのプロセスしか使っていないので、DummyVecEnv.

>We chose the MlpPolicy because input of CartPole is a feature vector, not images.

CartPoleの入力は画像ではなく、特徴ベクトルなので、MlpPolicyを選択しました。

>The type of action to use (discrete/continuous) will be automatically deduced from the environment action space

使用するアクションの種類（離散/連続）は、環境のアクション空間から自動的に推測されます。


In [ ]:
env = gym.make('LunarLander-v2')

## Create model

In [ ]:
model = DQN('MlpPolicy', env, learning_rate=1e-3, prioritized_replay=True, verbose=1)

We create a helper function to evaluate the agent:

## Define evaluate

In [ ]:
def evaluate(model, num_steps=1000):
  """
  Evaluate a RL agent
  :param model: (BaseRLModel object) the RL Agent
  :param num_steps: (int) number of timesteps to evaluate it
  :return: (float) Mean reward for the last 100 episodes
  """
  episode_rewards = [0.0]
  obs = env.reset()
  for i in range(num_steps):
      # _states are only useful when using LSTM policies
      action, _states = model.predict(obs)

      obs, reward, done, info = env.step(action)
      
      # Stats
      episode_rewards[-1] += reward
      if done:
          obs = env.reset()
          episode_rewards.append(0.0)
  # Compute mean reward for the last 100 episodes
  mean_100ep_reward = round(np.mean(episode_rewards[-100:]), 1)
  print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))
  
  return mean_100ep_reward

>Let's evaluate the un-trained agent, this should be a random agent.

訓練されていないエージェントを評価しましょう。これはランダムなエージェントであるべきです。

In [ ]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_steps=10000)

Mean reward: -629.5 Num episodes: 112


ランダムなので相当結果悪いですねwww

## Train the agent and save it (I)

まずは10000回学習させてみます．

In [ ]:
# Train the agent
model.learn(total_timesteps=10000, log_interval=10)
# Save the agent
model.save("dqn_lunar10000")
del model  # delete trained model to demonstrate loading

--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 10       |
| mean 100 episode reward | -358     |
| steps                   | 920      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 20       |
| mean 100 episode reward | -311     |
| steps                   | 2379     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 30       |
| mean 100 episode reward | -239     |
| steps                   | 6428     |
--------------------------------------


## Load the trained agent

10000回学習させたモデルをロードしてみます．

In [ ]:
model = DQN.load("dqn_lunar10000")

ランダムよりはマシになってますね．

In [ ]:
# Evaluate the trained agent
mean_reward = evaluate(model, num_steps=10000)

Mean reward: -74.1 Num episodes: 11


## Train the agent and save it (II)

モデルに学習環境をセットします．
こいつの存在に辿り着くまでに結構時間がかかりました．

In [ ]:
model.set_env(env)

ここからは先ほどと同じですね．追加で学習するので，ファイル名は20000にしておきます．

In [ ]:
# Train the agent
model.learn(total_timesteps=10000, log_interval=10)
# Save the agent
model.save("dqn_lunar20000")
del model  # delete trained model to demonstrate loading

--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 10       |
| mean 100 episode reward | -61.2    |
| steps                   | 1821     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 20       |
| mean 100 episode reward | -63.5    |
| steps                   | 9977     |
--------------------------------------


20000回学習させたモデルをロードしてみます．

In [ ]:
model = DQN.load("dqn_lunar20000")

いい感じでちゃんと学習が引き継がれているのではないでしょうか．

In [ ]:
# Evaluate the trained agent
mean_reward = evaluate(model, num_steps=10000)

Mean reward: -22.3 Num episodes: 14
